In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

tf.__version__
keras.__version__

'2.7.0'

In [2]:
data_name = "Data/Datafinal.csv"
target_name = "Data/Targetfinal.csv"

data = pd.read_csv(data_name)
target = pd.read_csv(target_name)

fs = 200
#turn header into time stamp
newData = (data.T.reset_index().T.reset_index(drop=True).set_axis([f'{i/fs}' for i in range(data.shape[1])], axis=1))
newTarget = (target.T.reset_index().T.reset_index(drop=True).set_axis([f'{i/fs}' for i in range(target.shape[1])], axis=1))

#print(newData)
#print(newTarget)

In [3]:
#replace row 0 with correct target data

newTarget.iloc[0,:] = np.zeros(newTarget.shape[1])
newTarget.iloc[0,0:1200] = np.ones(1200)
newTarget.iloc[0,15613:16813] = np.ones(1200)
newTarget.iloc[0,31226:32426] = np.ones(1200)
newTarget.iloc[0,46839:48039] = np.ones(1200)

newTarget.head()
#print(newTarget)

,0.0,0.005,0.01,0.015,0.02,0.025,0.03,0.035,0.04,0.045,...,312.21,312.215,312.22,312.225,312.23,312.235,312.24,312.245,312.25,312.255
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#turn target index into number

targetData = np.zeros(newTarget.shape[1])

for i in range(newTarget.shape[1]):
    for j in range(newTarget.shape[0]):
        if newTarget.iloc[j,i] != 0:
            targetData[i] = 1+j

print(targetData)

[ 1.  1.  1. ... 13. 13. 13.]


In [5]:
#split train test
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

trainData, testData, trainTarget, testTarget = train_test_split(np.transpose(newData), np.transpose(targetData), test_size=.2,random_state=0)

print(trainData.shape)
print(testData.shape)
print(trainTarget.shape)
print(testTarget.shape)

(49961, 96)
(12491, 96)
(49961,)
(12491,)


In [6]:
#convert string to float

trainDataNumeric = trainData.to_numpy()
testDataNumeric = testData.to_numpy()

for i in range(trainDataNumeric.shape[0]):
    for j in range(trainDataNumeric.shape[1]):
        if type(trainDataNumeric[i,j]) == str:
            trainDataNumeric[i,j] = trainDataNumeric[i,j][:-3]

for i in range(testDataNumeric.shape[0]):
    for j in range(testDataNumeric.shape[1]):
        if type(testDataNumeric[i,j]) == str:
            testDataNumeric[i,j] = testDataNumeric[i,j][:-3]

trainDataFloat = trainDataNumeric.astype(np.float)
testDataFloat = testDataNumeric.astype(np.float)

In [14]:
#NN model

input = keras.layers.Input(shape=[96])
flat = keras.layers.Flatten(input_shape=[96])(input)
batch1=keras.layers.BatchNormalization()(flat)
hidden1 = keras.layers.Dense(420, activation="relu")(batch1)
drop1=keras.layers.Dropout(0.2)(hidden1)
hidden2 = keras.layers.Dense(140, activation="relu")(drop1)
concat = keras.layers.Concatenate()([flat, hidden2])
output = keras.layers.Dense(14, activation="softmax")(concat)
model = keras.Model(inputs=[input], outputs=[output])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 96)]         0           []                               
                                                                                                  
 flatten_1 (Flatten)            (None, 96)           0           ['input_2[0][0]']                
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 96)          384         ['flatten_1[0][0]']              
 rmalization)                                                                                     
                                                                                                  
 dense_3 (Dense)                (None, 420)          40740       ['batch_normalization_1[0][

In [15]:
#Train NN

model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=['accuracy'])

with tf.device("/device:GPU:1"):
    model = model.fit(trainDataFloat, trainTarget, epochs=500, validation_data=(testDataFloat, testTarget))

Epoch 1/500
1562/1562 [==============================] - 4s 2ms/step - loss: 2.3598 - accuracy: 0.2914 - val_loss: 2.3641 - val_accuracy: 0.3142
Epoch 2/500
1562/1562 [==============================] - 3s 2ms/step - loss: 1.9540 - accuracy: 0.3692 - val_loss: 2.5057 - val_accuracy: 0.3552
Epoch 3/500
1562/1562 [==============================] - 3s 2ms/step - loss: 1.7723 - accuracy: 0.4152 - val_loss: 3.6040 - val_accuracy: 0.3752
Epoch 4/500
1562/1562 [==============================] - 3s 2ms/step - loss: 1.6343 - accuracy: 0.4495 - val_loss: 2.0459 - val_accuracy: 0.4495
Epoch 5/500
1562/1562 [==============================] - 3s 2ms/step - loss: 1.5010 - accuracy: 0.4799 - val_loss: 1.2053 - val_accuracy: 0.5544
Epoch 6/500
1562/1562 [==============================] - 3s 2ms/step - loss: 1.4132 - accuracy: 0.5063 - val_loss: 1.6182 - val_accuracy: 0.5018
Epoch 7/500
1562/1562 [==============================] - 3s 2ms/step - loss: 1.3325 - accuracy: 0.5310 - val_loss: 1.3767 - val_ac

In [16]:
model.history['accuracy'][-1] #500  epochs

0.9839274883270264